In [ ]:
from datetime import datetime, date
from pathlib import PurePosixPath
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round

# Configuração
CATALOG = "workspace"
SCHEMA  = "default"
VOLUME  = "elt_volume"

VOL_ROOT = PurePosixPath("/Volumes") / CATALOG / SCHEMA / VOLUME
SILVER_ROOT = VOL_ROOT / "silver"
today_str = date.today().isoformat()
time_str  = datetime.now().strftime("%H%M")  # Current time: 1503
SILVER_PART = SILVER_ROOT / today_str.replace("-", "/")
SILVER_BASE = SILVER_PART / f"vendas_silver_processed_{time_str}"

spark = SparkSession.builder.getOrCreate()

# Recupera caminho do bronze_csv da task anterior
task_key = "Execute_process_bronze_clean"  # Nome real da task no Workflow
try:
    BRONZE_CSV = dbutils.jobs.taskValues.get(taskKey=task_key, key="bronze_csv", debugValue="Not set")
    if not BRONZE_CSV or not dbutils.fs.ls(str(BRONZE_CSV)):
        raise FileNotFoundError(f"Arquivo não encontrado em {BRONZE_CSV}")
except Exception as e:
    # Fallback: Procurar o diretório mais recente criado pelo Bronze
    BRONZE_ROOT = VOL_ROOT / "bronze"
    bronze_base = str(BRONZE_ROOT / today_str.replace("-", "/"))
    bronze_files = dbutils.fs.ls(bronze_base)
    BRONZE_CSV = None
    if bronze_files:
        bronze_dirs = sorted([f for f in bronze_files if f.isDir() and f.name.startswith("vendas_bronze_cleaned_")], key=lambda x: x.modificationTime, reverse=True)
        for sub_dir_entry in bronze_dirs:
            sub_dir = sub_dir_entry.path.replace("dbfs:", "")
            sub_files = dbutils.fs.ls(sub_dir)
            for sf in sub_files:
                if sf.name.endswith(".csv"):
                    BRONZE_CSV = sf.path.replace("dbfs:", "")
                    break
            if BRONZE_CSV:
                break
    if not BRONZE_CSV or not dbutils.fs.ls(str(BRONZE_CSV)):
        raise FileNotFoundError(f"Arquivo não encontrado em subdiretórios de {bronze_base}")

# Cria diretórios necessários
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")
dbutils.fs.mkdirs(str(SILVER_PART))

# Leitura do CSV da camada bronze
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(f"dbfs:{BRONZE_CSV}"))

# Processamento: Geração de campos essenciais
df_silver = (df
    .withColumn("valor_liquido", round(col("preco_unit") * col("qtd") * (1 - col("desconto") / 100), 2))
    .withColumn("score_venda", col("valor_liquido") / 1000)  # Exemplo de pontuação
    .withColumn("flag_anomalia", (col("valor_liquido") > 5000).cast("integer")))

# Seleção de campos essenciais
essential_columns = ["venda_id", "data", "vendedor_id", "produto_id", "categoria", "valor_liquido", "desconto", "score_venda", "flag_anomalia"]
df_silver = df_silver.select(essential_columns)

# Contagem de linhas processadas
total_rows_processed = df_silver.count()

# Salva o arquivo processado
df_silver.coalesce(1).write.mode("overwrite").option("header", "true").csv(f"dbfs:{SILVER_BASE}")

# Encontra o primeiro arquivo particionado para task value
silver_files = dbutils.fs.ls(f"dbfs:{SILVER_BASE}")
if not silver_files:
    raise FileNotFoundError(f"Nenhum arquivo encontrado em {SILVER_BASE}")
silver_csv_path = [f.path.replace("dbfs:", "") for f in silver_files if f.path.endswith(".csv")][0]

# Registro de metadados
metadata = {
    "total_rows_processed": total_rows_processed,
    "output_path": silver_csv_path
}

# Salva metadados como JSON
metadata_path = SILVER_PART / f"metadata_vendas_silver_{time_str}.json"
dbutils.fs.put(f"dbfs:{metadata_path}", str(metadata), overwrite=True)

# Salva contextos para próximas tasks
try:
    dbutils.jobs.taskValues.set(key="silver_csv", value=silver_csv_path)
    dbutils.jobs.taskValues.set(key="silver_metadata", value=str(metadata_path))
except Exception as e:
    raise